In [1]:
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import numpy as np
import import_ipynb
import data_structures as ds
from datetime import datetime
import psycopg2
from psycopg2.extras import execute_values

importing Jupyter notebook from data_structures.ipynb


In [2]:
def ck_pull_store():
    page = requests.get(r'https://www.cardkingdom.com/purchasing/mtg_singles?filter%5Bipp%5D=100&filter%5Bsort%5D'
                            r'=name&filter%5Bsearch%5D=mtg_advanced&filter%5Bname%5D=&filter%5Bcategory_id%5D=0'
                            r'&filter%5Bnonfoil%5D=1&filter%5Brarity%5D%5B0%5D=M&filter%5Brarity%5D%5B1%5D=R&filter'
                            r'%5Brarity%5D%5B2%5D=U&filter%5Brarity%5D%5B3%5D=C&filter%5Brarity%5D%5B5%5D=S&filter'
                            r'%5Bprice_op%5D=%3C%3D&filter%5Bprice%5D=499.99&page=1')

    soup = BeautifulSoup(page.content, 'html.parser')

    pg_num = int(list(soup.children)[-31].get_text())

    print('CK page number = ' + str(pg_num))

    items = []
    for n in range(pg_num):
        print(n)
        count = 0
        page = requests.get(r'https://www.cardkingdom.com/purchasing/mtg_singles?filter%5Bipp%5D=100&filter%5Bsort%5D'
                            r'=name&filter%5Bsearch%5D=mtg_advanced&filter%5Bname%5D=&filter%5Bcategory_id%5D=0'
                            r'&filter%5Bnonfoil%5D=1&filter%5Brarity%5D%5B0%5D=M&filter%5Brarity%5D%5B1%5D=R&filter'
                            r'%5Brarity%5D%5B2%5D=U&filter%5Brarity%5D%5B3%5D=C&filter%5Brarity%5D%5B5%5D=S&filter'
                            r'%5Bprice_op%5D=%3C%3D&filter%5Bprice%5D=499.99&page=' + str(n + 1))

        soup = BeautifulSoup(page.content, 'html.parser')

        cards_raw = [c for c in list(soup.children) if len(str(c)) > 10][:-13]
        cards_raw = [b for b in cards_raw if 'sellDollarAmount' in str(b)]

        for card in cards_raw:
            card_1 = list(card.children)[-1]
            if count == 0:
                card_2 = list(card_1.children)[-1]
                card_3 = list(card_2.children)[-2]
                card_4 = list(card_3.children)[-2]
                card_5 = list(card_4.children)[-3]
                card_6 = list(card_5.children)[-2]
                card_7 = list(card_6.children)[-2]
                card_8 = list(card_7.children)[-1]

                price_raw = list(card_8.children)[-1]
                info_raw = list(card_8.children)[-3]

                info_1 = list(info_raw.children)[1]
                info_2 = list(info_1.children)[1]
                name = list(info_2.children)[1].get_text()
                info_3 = list(info_2.children)[3]
                info_4 = list(info_3.children)[1]
                set = list(info_4.children)[0].split(' (')[0]
                if len(list(info_4.children)) > 1:
                    foil = True
                else:
                    foil = False

                price_1 = list(price_raw.children)[3]
                price_2 = list(price_1.children)[3]
                price_3 = list(price_2.children)[1]
                price_4 = list(price_3.children)[1]
                price_5 = list(price_4.children)[0]
                dollars = list(price_5.children)[1].get_text()
                cents = list(price_5.children)[2].get_text()
                price = float(dollars) + float(cents) * .01
            else:
                info_raw = list(card_1.children)[-3]
                info_1 = list(info_raw.children)[1]
                info_2 = list(info_1.children)[1]
                name = list(info_2.children)[1].get_text()
                info_3 = list(info_2.children)[3]
                info_4 = list(info_3.children)[1]
                set = list(info_4.children)[0].split(' (')[0]
                if len(list(info_4.children)) > 1:
                    foil = True
                else:
                    foil = False

                price_raw = list(card_1.children)[-1]
                price_1 = list(price_raw.children)[3]
                price_2 = list(price_1.children)[3]
                price_3 = list(price_2.children)[1]
                price_4 = list(price_3.children)[1]
                price_5 = list(price_4.children)[0]
                dollars = list(price_5.children)[1].get_text()
                cents = list(price_5.children)[2].get_text()
                price = float(dollars) + float(cents) * .01

            items.append([name, foil, set, price])
            count += 1

    df = pd.DataFrame(items, columns=['name', 'foil', 'set', 'price'])
    df = df[df['foil'] == False]
    df['uuid'] = np.nan

    conn = psycopg2.connect(host="localhost", port=5432, database="", user="", password="")
    cur = conn.cursor()
    
    # So that it stops trying to convert to float
    df['uuid']=df['uuid'].astype(str)
    
    c_get_u_mistakes = 0
    for index, row in df.iterrows():
        try:
            uuid = ds.get_uuid(cur, row['name'], row['set'], 'ck')
            df.at[index, 'uuid'] = uuid
        except Exception as e:
            c_get_u_mistakes += 1
    print('Getting uuid mistakes: ' + str(c_get_u_mistakes))

    df = df.loc[df['uuid'] != 'nan']
    
    pd.set_option('display.max_columns', None)
    df = df[pd.notnull(df['uuid'])]
    df['ckdate'] = datetime.now()
    
    df.drop_duplicates(subset=['uuid', 'ckdate'], inplace=True)
    
    t_df = tuple(df[['uuid', 'ckdate', 'price']].itertuples(index=False, name=None))
    execute_values(cur, "INSERT INTO ckbuydata (uuid, ckdate, ckbuyprice) VALUES %s", t_df)
    conn.commit()
    
    cur.close()
    conn.close()

In [3]:
def main():
    start = time.time()
    ck_pull_store()
    end = time.time()
    print(end - start)

In [4]:
if __name__ == '__main__':
    main()

CK page number = 218
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
Getting uuid mistakes: 6382
751.051374912262
